## Dependencies

In [76]:
import numpy as np
# BatchSize, ChannelsInFinalConvLayer, ImageWidth?, ImageHeight? 
t = torch.ones((376, 11, 2, 2))
print(t)
t.size()
t = t.view(376*2*2, 11)
print(t)
t.size()


( 0 , 0 ,.,.) = 
  1  1
  1  1

( 0 , 1 ,.,.) = 
  1  1
  1  1

( 0 , 2 ,.,.) = 
  1  1
  1  1
    ... 

( 0 , 8 ,.,.) = 
  1  1
  1  1

( 0 , 9 ,.,.) = 
  1  1
  1  1

( 0 ,10 ,.,.) = 
  1  1
  1  1
      ⋮  

( 1 , 0 ,.,.) = 
  1  1
  1  1

( 1 , 1 ,.,.) = 
  1  1
  1  1

( 1 , 2 ,.,.) = 
  1  1
  1  1
    ... 

( 1 , 8 ,.,.) = 
  1  1
  1  1

( 1 , 9 ,.,.) = 
  1  1
  1  1

( 1 ,10 ,.,.) = 
  1  1
  1  1
      ⋮  

( 2 , 0 ,.,.) = 
  1  1
  1  1

( 2 , 1 ,.,.) = 
  1  1
  1  1

( 2 , 2 ,.,.) = 
  1  1
  1  1
    ... 

( 2 , 8 ,.,.) = 
  1  1
  1  1

( 2 , 9 ,.,.) = 
  1  1
  1  1

( 2 ,10 ,.,.) = 
  1  1
  1  1
...     
      ⋮  

(373, 0 ,.,.) = 
  1  1
  1  1

(373, 1 ,.,.) = 
  1  1
  1  1

(373, 2 ,.,.) = 
  1  1
  1  1
    ... 

(373, 8 ,.,.) = 
  1  1
  1  1

(373, 9 ,.,.) = 
  1  1
  1  1

(373,10 ,.,.) = 
  1  1
  1  1
      ⋮  

(374, 0 ,.,.) = 
  1  1
  1  1

(374, 1 ,.,.) = 
  1  1
  1  1

(374, 2 ,.,.) = 
  1  1
  1  1
    ... 

(374, 8 ,.,.) = 
  1  1
  1  1

(374, 9 ,

torch.Size([1504, 11])

In [1]:
import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torchvision.models as models

import imp
import os
import sys
import math
import utils.training as train_utils; imp.reload(train_utils)
import utils.plot as plot_utils; imp.reload(plot_utils)
import time
from tqdm import tqdm

In [2]:
DATA_PATH='data/'
RESULTS_PATH='results/'
WEIGHTS_PATH='models/'
PROJECT_NAME='tiramisu'

## Design

**FirstConvLayer**

* 3x3 Conv2D (pad=, stride=, in_chans=3, out_chans=48)

**DenseLayer**

* BatchNorm
* ReLU
* 3x3 Conv2d (pad=, stride=, in_chans=, out_chans=) - "no resolution loss" - padding included
* Dropout (.2)

**DenseBlock**

* Input = FirstConvLayer, TransitionDown, or TransitionUp
* Loop to create L DenseLayers (L=n_layers)
* On TransitionDown we Concat(Input, FinalDenseLayerActivation)
* On TransitionUp we do not Concat with input, instead pass FinalDenseLayerActivation to TransitionUp block

**TransitionDown**

* BatchNorm
* ReLU
* 1x1 Conv2D (pad=, stride=, in_chans=, out_chans=)
* Dropout (0.2)
* 2x2 MaxPooling

**Bottleneck**

* DenseBlock (15 layers)

**TransitionUp**

* 3x3 Transposed Convolution (pad=, stride=2, in_chans=, out_chans=)
* Concat(PreviousDenseBlock, SkipConnection) - from cooresponding DenseBlock on transition down

**FinalBlock**

* 1x1 Conv2d (pad=, stride=, in_chans=256, out_chans=n_classes)
* Softmax

**FCDenseNet103 Architecture**

* input (in_chans=3 for RGB)
* 3x3 ConvLayer (out_chans=48)
* DB (4 layers) + TD
* DB (5 layers) + TD
* DB (7 layers) + TD
* DB (10 layers) + TD
* DB (12 layers) + TD
* Bottleneck (15 layers)
* TU + DB (12 layers)
* TU + DB (10 layers)
* TU + DB (7 layers)
* TU + DB (5 layers)
* TU + DB (4 layers)
* 1x1 ConvLayer (out_chans=n_classes) n_classes=11 for CamVid
* Softmax

**FCDenseNet56**

GrowthRate (k) = 12
4 layers per dense block
1 Conv Layer
5 DenseBlocks Downsample (20 layers)
5 TransitionDown
4 Bottleneck layers
5 Dense Blocks Upsample (20 layers)
5 TransitionUp
1 Conv Layer
1 Softmax layer (doesn't count)
56 Total layers

## Layers

In [72]:
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.out_channels = in_channels + growth_rate
        self.add_module('norm', nn.BatchNorm2d(num_features=in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        
        #author's impl - lasange 'same' pads with half filter size (rounded down) on "both" sides
        self.add_module('conv', nn.Conv2d(in_channels=in_channels, 
                out_channels=self.out_channels, kernel_size=3, stride=1, 
                  padding=3//2, bias=True))
        
        self.add_module('drop', nn.Dropout2d(0.2))

    def forward(self, x):
        out = self.features(x)
        return torch.cat([x, new_features], 1) # 1 = channel axis
    

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers):
        super(DenseBlock, self).__init__()
        
        n_channels = in_channels
        for i in range(n_layers):
            layer = DenseLayer(n_channels, growth_rate)
            self.add_module('denselayer%d' % (i + 1), layer)
            n_channels += growth_rate
    
    def forward(self, x):
        return self.features(x)

    
class TransitionDown(nn.Module):
    def __init__(self, in_channels):
        super(TransitionDown, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_features=in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        #what is out_channels?
        self.add_module('conv', nn.Conv2d(in_channels=in_channels, out_channels=in_channels,
                                          kernel_size=1, stride=1, padding=0, bias=True))
        self.add_module('drop', nn.Dropout2d(0.2))
        self.add_module('maxpool', nn.MaxPool2d(2))
        
    def forward(self, x):
        return self.forward(x)
    
class TransitionUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionUp, self).__init__()
        self.add_module('transpose', nn.ConvTranspose2d(in_channels=in_channels, 
                           out_channels=out_channels, kernel_size=3, stride=2, 
                            padding=0, bias=True))
        
    def forward(self, x, skip_connection):
        out = self.forward(x)
        #l = ConcatLayer([l, skip_connection], cropping=[None, None, 'center', 'center'])
        return torch.cat([out, skip_connection])

class Bottleneck(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers):
        super(Bottleneck, self).__init__()
        self.add_module('bottleneck', DenseBlock(in_channels, growth_rate, n_layers))

    def forward(self, x):
        return self.forward(x)

## Model

In [78]:
class FCDenseNet(nn.Module):
    def __init__(self, in_channels=3, n_blocks=5, layers_per_block=5, growth_rate=16, 
                 out_chans_first_conv=48, n_classes=11):
        super(FCDenseNet, self).__init__()
        self.n_blocks = n_blocks
        self.n_channels = 0
        
        #####################
        # First Convolution #
        #####################
        #W=HeUniform(gain='relu') ??
        #pad='same'
        self.firstConv = nn.Conv2d(in_channels=in_channels, 
                  out_channels=out_chans_first_conv, kernel_size=3, 
                  stride=1, padding=in_channels//2, bias=False)
        self.n_channels += out_chans_first_conv
        
        #####################
        # Downsampling path #
        #####################
        
        skipConnectChannels = []
        self.denseBlocksDown = []
        self.transDownBlocks = []
        for i in range(n_blocks):
            db = DenseBlock(self.n_channels, growth_rate, layers_per_block)
            self.add_module("DBDown%d" % (i+1), db)
            self.denseBlocksDown.append(db)
            self.n_channels += (growth_rate*layers_per_block)
            skipConnectChannels.insert(0, self.n_channels)
            
            td = TransitionDown(self.n_channels)
            self.transDownBlocks.append(td)
            self.add_module("TD%d" % (i+1), td)
            
        #####################
        #     Bottleneck    #
        #####################
        
        self.bottleneck = Bottleneck(self.n_channels, growth_rate, layers_per_block)
        prev_block_channels = growth_rate*layers_per_block
        self.n_channels += prev_block_channels 
        
        #######################
        #   Upsampling path   #
        #######################

        self.transUpBlocks = []
        self.denseBlocksUp = [] 
        for i in range(n_blocks):
            tu = TransitionUp(self.n_channels, prev_block_channels + skipConnectChannels[i])
            self.transUpBlocks.append(td)
            self.add_module("TU%d" % (i+1), tu)
            
            self.n_channels = prev_block_channels + skipConnectChannels[i]

            db = DenseBlock(self.n_channels, growth_rate, layers_per_block)
            self.denseBlocksUp.append(db)
            self.add_module("DBUp%d" % (i+1), db)

            prev_block_channels = growth_rate*layers_per_block
            self.n_channels += prev_block_channels
            
        #####################
        #      Softmax      #
        #####################
        
        self.finalConv = nn.Conv2d(in_channels=self.n_channels, out_channels=n_classes,
                       kernel_size=1, stride=1, padding=0, bias=True)
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        out = self.firstConv(x)
        skip_connections = []
        for i in range(self.n_blocks):
            out = self.denseBlocksDown[i](out)
            skip_connections.insert(0, out)
            out = self.transDownBlocks[i](out)
            
        out = self.bottleneck(out)
        
        for i in range(self.n_blocks):
            out = self.transDownBlocks[i](out, skip_connections[i]) 
            out = self.denseBlocksUp[i](out)
        
        out = self.finalConv(out)
        # Reshape
        batch_size = out.size()[0]
        rows = out.size()[2]
        cols = out.size()[3]
        out = out.view(batch_size*row*cols, n_classes)
        
        out = self.softmax(out)
        return out

In [79]:
net = FCDenseNet()

In [80]:
print(net)

FCDenseNet (
  (firstConv): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (DBDown1): DenseBlock (
    (denselayer1): DenseLayer (
      (norm): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (drop): Dropout2d (p=0.2)
    )
    (denselayer2): DenseLayer (
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv): Conv2d(64, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (drop): Dropout2d (p=0.2)
    )
    (denselayer3): DenseLayer (
      (norm): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv): Conv2d(80, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (drop): Dropout2d (p=0.2)
    )
    (denselayer4): DenseLayer (
      (norm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)

## Data

In [76]:
BATCH_SIZE=64
CIFAR10_PATH=DATA_PATH+'cifar10/'

In [77]:
torch.cuda.manual_seed(1)

normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)

trainTransform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normTransform
])
testTransform = transforms.Compose([
    transforms.ToTensor(),
    normTransform
])

kwargs = {'num_workers': 1, 'pin_memory': True}
trainLoader = DataLoader(
    dset.CIFAR10(root=CIFAR10_PATH, train=True, download=True,
                 transform=trainTransform),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)
testLoader = DataLoader(
    dset.CIFAR10(root=CIFAR10_PATH, train=False, download=True,
                 transform=testTransform),
    batch_size=BATCH_SIZE, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


## Train

**Hyperparameters**

* WeightInitialization = HeUniform
* Optimizer = RMSProp
* LR = .001 with exponential decay of 0.995 after each epoch
* Data Augmentation = Random Crops, Vertical Flips
* ValidationSet with early stopping based on IoU or MeanAccuracy with patience of 100 (50 during finetuning)
* WeightDecay = .0001
* Finetune with full-size images, LR = .0001
* Dropout = 0.2
* BatchNorm "we use current batch stats at training, validation, and test time"

**CamVid**

* TrainingSet = 367 frames
* ValidationSet = 101 frames
* TestSet = 233 frames
* Images of resolution 360x480
* Images "Cropped" to 224x224 for training --- center crop?
* FullRes images used for finetuning
* NumberOfClasses = 11 (output)
* BatchSize = 3

**FCDenseNet103**

* GrowthRate = 16 (k, number of filters to each denselayer adds to the ever-growing concatenated output)
* No pretraining

In [82]:
existing_weights_fpath=None
nEpochs=1

net = DenseNet(growthRate=12, depth=40, reduction=1.0, 
                   bottleneck=False, nClasses=10)
net = net.cuda()

optimizer = optim.SGD(net.parameters(), lr=1e-1,
                momentum=0.9, weight_decay=1e-4)

print('  + Number of params: {}'.format(
    sum([p.data.nelement() for p in net.parameters()])))

if existing_weights_fpath:
    startEpoch = train_utils.load_weights(net, existing_weights_fpath)
    endEpoch = startEpoch + nEpochs
    print ('Resume training at epoch: {}'.format(startEpoch))
    if os.path.exists(RESULTS_PATH+'train.csv'): #assume test.csv exists
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not
    trainF = open(os.path.join(RESULTS_PATH, 'train.csv'), append_write)
    testF = open(os.path.join(RESULTS_PATH, 'test.csv'), append_write)
else:
    print ("Training new model from scratch")
    startEpoch = 1
    endEpoch = nEpochs
    trainF = open(os.path.join(RESULTS_PATH, 'train.csv'), 'w')
    testF = open(os.path.join(RESULTS_PATH, 'test.csv'), 'w')


for epoch in tqdm(range(startEpoch, endEpoch+1)):
    since = time.time()
    train_utils.adjust_opt("sgd", optimizer, epoch)
    train_utils.train(epoch, net, trainLoader, optimizer, trainF)
    train_utils.test(epoch, net, testLoader, optimizer, testF)
    time_elapsed = time.time() - since  
    print('Time {:.0f}m {:.0f}s\n'.format(
        time_elapsed // 60, time_elapsed % 60))
    if epoch != 1:
        os.system('./utils/plot.py {} &'.format(RESULTS_PATH))

trainF.close()
testF.close()


  0%|          | 0/1 [00:00<?, ?it/s]

  + Number of params: 1059298
Training new model from scratch


Epoch 1: Train - Loss: 1.370707	Error: 43.750000


100%|██████████| 1/1 [01:12<00:00, 72.19s/it]

Test - Loss: 1.3404, Error: 4590/10000 (46%)
Time 1m 12s

